**Автор: Анна Смелова**

## Решение домашнего задания к уроку 8 "Снижение размерности данных"

### Задание 1
#### Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
# функция для подсчета точности как доли правильных ответов
def get_accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [3]:
# масштабирование
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [4]:
# ковариация
def covariance(x, y):
    return np.sum(x * y) / (len(x) - 1)

In [5]:
# Класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [6]:
# Класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [7]:
# Класс дерева
class Tree:
    
    def __init__(self, max_depth=100): # максимальная глубина дерева
            
        self.max_depth = max_depth
        self.nodes = []
        self.leaves = []
        self.depth = 0
        self.tree = None
    
    # Расчет критерия Джини
    def gini(self, labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2

        return impurity
    
    # Расчет прироста
    def gain(self, left_labels, right_labels, root_gini):
        # доля выборки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

        return root_gini - p * self.gini(left_labels) - (1 - p) * self.gini(right_labels)
    
    # Разбиение датасета в узле
    def split(self, data, labels, column_index, t):
        left = np.where(data[:, column_index] <= t)
        right = np.where(data[:, column_index] > t)

        true_data = data[left]
        false_data = data[right]

        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels
    
    # Реализуем генерацию подмножества признаков для нахождения разбиения в узле
    def get_subsample(self, len_sample):
        # будем сохранять не сами признаки, а их индексы
        sample_indexes = list(range(len_sample))

        len_subsample = int(np.sqrt(len_sample))

        subsample = np.random.choice(sample_indexes, size=len_subsample, replace=False)

        return subsample
    
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):
        #  обозначим минимальное количество объектов в узле
        min_leaf_samples = 5

        root_gini = self.gini(labels)

        best_gain = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        feature_subsample_indices = self.get_subsample(n_features) # выбираем случайные признаки

        for index in feature_subsample_indices:
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([obj[index] for obj in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf_samples or len(false_data) < min_leaf_samples:
                    continue

                current_gain = self.gain(true_labels, false_labels, root_gini)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_gain > best_gain:
                    best_gain, best_t, best_index = current_gain, t, index

        return best_gain, best_t, best_index
    
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels):
        gain, t, index = self.find_best_split(data, labels)
        
        #  Базовый случай 1 - прекращаем рекурсию, когда нет прироста в качества
        if gain == 0:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)
 
        #  Базовый случай 2 - прекращаем рекурсию, когда достигли максимальной глубины дерева
        if self.depth >= self.max_depth:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)

        self.depth += 1
        
        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        self.nodes.append(Node(index, t, true_branch, false_branch))
        return Node(index, t, true_branch, false_branch)
    
    def classify_object(self, obj, node):
        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)
    
    def fit(self, data, labels):
        self.tree = self.build_tree(data, labels)
        return self
    
    def predict(self, data):
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.tree)
            classes.append(prediction)
        return classes

In [8]:
# Класс случайного леса
class RandomForest:
    def __init__(self, n_trees, # количествоо деревьев 
                 max_depth=100, # максимальная глубина деревьев
                 max_leaves=200, # максимальное количество листьев в деревьях
                 min_leaf_objects=1):  # минимальное количество объектов в листе
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.max_leaves = max_leaves
        self.min_leaf_objects = min_leaf_objects
        self.forest = [] 
        self.train_accuracy = []
        self.test_accuracy = []
        self.train_roc = []
        self.test_roc = []
    
    # Реализуем генерацию 𝑁 бутстрап-выборок
    def get_bootstrap(self, data, labels, N):
        n_samples = data.shape[0] # размер совпадает с исходной выборкой
        bootstrap = []

        for i in range(N):
            sample_index = np.random.randint(0, n_samples, size=n_samples)
            b_data = data[sample_index]
            b_labels = labels[sample_index]

            bootstrap.append((b_data, b_labels))

        return bootstrap
    
    def fit(self, data, labels):
        forest = []
        bootstrap = self.get_bootstrap(data, labels, self.n_trees)

        for b_data, b_labels in bootstrap:
            new_tree = Tree(self.max_depth)
            new_tree.fit(b_data, b_labels)
            self.forest.append(new_tree)
            
        return self

    # предсказание голосованием деревьев
    def tree_vote(self, data):
        # добавим предсказания всех деревьев в список
        predictions = []
        for tree in self.forest:
            
            tree_vote = tree.predict(data)
            predictions.append(tree_vote)

        # сформируем список с предсказаниями для каждого объекта
        predictions_per_object = list(zip(*predictions))

        # выберем в качестве итогового предсказания для каждого объекта то,
        # за которое проголосовало большинство деревьев
        voted_predictions = []
        for obj in predictions_per_object:
            voted_predictions.append(max(set(obj), key=obj.count))

        return voted_predictions

In [9]:
# загрузим датасет iris
iris = datasets.load_iris()
data = iris.data
target = iris.target
target_names = iris.target_names

In [10]:
# основная информация по датасету
print(data.shape, target.shape)
print(target_names)
display(data[:5,:], target[:5])

(150, 4) (150,)
['setosa' 'versicolor' 'virginica']


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

array([0, 0, 0, 0, 0])

In [11]:
# масштабируем
data = standard_scale(data)
display(data[:5,:])

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [12]:
# перемешивание датасета
np.random.seed(0)
shuffle_index = np.random.permutation(data.shape[0])
X, y = data[shuffle_index], target[shuffle_index]

# разделим датасет на обучающую и отложенную выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(105, 4) (45, 4) (105,) (45,)


In [13]:
# Обучим RandomForest до применения PCA
clf_before = RandomForest(n_trees=20)
clf_before.fit(X_train, y_train)
train_labels = clf_before.tree_vote(X_train)
print(f'Train accuracy {get_accuracy_metric(y_train, train_labels)}')

Train accuracy 98.09523809523809


In [14]:
# найдем собственные векторы и собственные значения
covariance_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979905, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.1045707202107, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569727, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292895, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [15]:
# оценим долю дисперсии, которая описывается найденными компонентами.
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329983, 22.85076178670178, 3.6689218892828785, 0.5178709107154825]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. В то же время последняя компонента описывает всего 0.5% и может быть отброжена без страха значительных потерь в качестве нашего анализа. Мы отбросим последние две компоненты, оставив первые две.

In [16]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [17]:
# Сформируем новую матрицу "объекты-признаки"
X_reduced = X.dot(W)
X_reduced[:5,:]

array([[ 1.4676452 ,  0.44227159],
       [ 0.56210831,  1.76472438],
       [-2.44617739, -2.15072788],
       [ 2.30243944, -0.42006558],
       [-2.23284716, -0.22314807]])

In [18]:
# разделим датасет на обучающую и отложенную выборки
X_train_reduced, X_valid_reduced, y_train_reduced, y_valid_reduced = train_test_split(X_reduced, y, test_size=0.3, random_state=42)
print(X_train_reduced.shape, X_valid_reduced.shape, y_train_reduced.shape, y_valid_reduced.shape)

(105, 2) (45, 2) (105,) (45,)


In [19]:
# Обучим RandomForest после применения PCA
clf_after = RandomForest(n_trees=20)
clf_after.fit(X_train_reduced, y_train_reduced)
train_labels_reduced = clf_after.tree_vote(X_train_reduced)
print(f'Train accuracy {get_accuracy_metric(y_train_reduced, train_labels_reduced)}')

Train accuracy 95.23809523809523


In [21]:
# точность на отложенной выборке
test_labels = clf_before.tree_vote(X_valid)
test_labels_reduced = clf_after.tree_vote(X_valid_reduced)
print(f'Valid accuracy before PCA: {get_accuracy_metric(y_valid, test_labels)}, Train: {get_accuracy_metric(y_train, train_labels)}')
print(f'Valid accuracy after PCA: {get_accuracy_metric(y_valid_reduced, test_labels_reduced)}, Train: {get_accuracy_metric(y_train_reduced, train_labels_reduced)}')

Valid accuracy before PCA: 91.11111111111111, Train: 98.09523809523809
Valid accuracy after PCA: 95.55555555555556, Train: 95.23809523809523


Вывод: после применения PCA качество классификации выросло. Также уменьшилось переобучение.